In [ ]:
"""
<영화 데이터 수집>
 - 수집처 : 메가박스
 - 수집처 URL : https://www.megabox.co.kr/movie
 - 수집 위치(웹 네비게이션) : 영화 > 전체영화 > 박스오피스
 - 수집 데이터 (10개 영화정보만 수집)
  : 영화제목, 실관람평점, 예매순위, 예매율, 누적관객수
  : 영화제목, 개별관람평점, 개별관란명내용, 긍정(관람평점6이상)/부정(관렴평점6미만)
"""

### 라이브러리 정의하기

In [1]:
### 웹브라우저 오픈을 위한 라이브러리
# - 브라우저 제어
# - 설치해야 합니다. : pip install selenium
from selenium import webdriver

### 데이터 처리
import pandas as pd

### 동적 웹페이지 처리 라이브러리
# - 동적 웹페이지 처리처럼 html.parser 처리를 하지 않아도,
#   라이브러리가 알아서 내부적으로 처리해줌
from selenium.webdriver.common.by import By

### 시간 라이브러리
# - 브라우저의 로딩시간이 길어지는 경우에는
#  -> 프로그램 처리 시간도 늦춰주어야 합니다.
#  -> time.sleep(초)를 이용해서 늦춰주면 됩니다.
import time

# ### 웹 프로토콜을 통해서 요청 URL을 전송하고 결과를 받아오는 라이브러리
# import requests

# ### 전달받은 태그 코드의 데이터를 검색 추출하기 위한 라이브러리
# from bs4 import BeautifulSoup

# ### 전달받은 태그 코드를 인식하지 못하는 경우를 대비해서 라이브러리 추가만 해놓기
# # - 설치 필요 : pip install lxml
# # - 설치 후 커널 재시작해야 합니다.
# from openpyxl.workbook import Workbook

### 크롬브라우저 제어하기

In [2]:
def getDriver() :
    return webdriver.Chrome()

In [3]:
### 크롬브라우저 open하기
driver = getDriver()

### 수집할 웹페이지 URL 정의, 요청 및 html 코드 수집

In [4]:
### 수집할 웹페이지 URL
url = "https://www.megabox.co.kr/movie"

### 브라우저를 통해서 용청 및 html 코드 받아오기
driver.get(url)

In [ ]:
"""
** selenium 동적페이지 처리시에는 
   -> 별도의 파싱(html.parser) 작업을 하지 않아도 됨...
   -> 바로 태그 경로를 이용해서 데이터 수집하면 됩니다.

** 태그 정보 조회 함수 : driver.find_elements(By.CSS_SELECTOR, "태그 경로 패턴")
                          -> 반환 타입은 list 타입 입니다.(여러 건 조회)

                       : driver.find_element(By.CSS_SELECTOR, "태그 경로 패턴")
                          -> 반환 타입은 html 태그 자체 입니다. (1건 조회)
                          
** 클릭 가능한 태그들 : a, input, button 태그 이름들은 클릭 가능한 태그들 입니다.
                      : a태그의 경우 href 속성 값에 #만 있는 경우에는 링크 주소가 없는 경우로,
                        상위 태그에서 링크(페이지 전환) 처리를 한다고 보면 됩니다.
"""

### 영화 목록 전체 추출하기

In [ ]:
"""
<처리 순서>
 1. 영화 목록 전체 정보를 추출
  -> 클릭 할 수 있는 태그를 찾아서 수집해야 합니다
 2. 추출한 각 영화 정보를 크릭하여 페이지 전환
 3. 전환된 페이지에서 해당 영화 정보() 추출
  : 영화제목, 실관람평점, 예매순위, 예매율, 누적관객수
  : 영화제목, 개별관람평점, 개별관란명내용, 긍정(관람평점6이상)/부정(관렴평점6미만)
 4. 전환된 페이지에서 추출이 끝나면 -> 다시 목록으로 페이지 전환
 5. 다음 영화 정보 클릭하여 페이지 전화하면서 반복 처리
"""

In [5]:
# 1. 영화 목록 전체 정보를 추출
#  -> 클릭 할 수 있는 태그를 찾아서 수집해야 합니다
# #movieList > li > div.movie-list-info
# div.movie-list-info

### 영화 목록 전체 가지고 오기 위한 태그 경로 정의
movie_list_path = "div.movie-list-info"

### 태그 경로에 대한 태그 정보 조회하기
# - find_elements 또는 find_element의 조회 결과 내에는 태그가 안보임
#   (각 태그의 메모리 주소 정보만 가지고 있음)
movie_elements = driver.find_elements(By.CSS_SELECTOR, "div.movie-list-info")
print(len(movie_elements))

20


### 20개 태그 정보(list 타입) 중에, 0번째 인덱스의 태그 정보로 샘플링하기

In [6]:
### 클릭 테스트 하기
# - 0번째 태그정보를 이용해서 실제 클릭 후 페이지 전환이 잘 되는지 확인하기
# - 클릭(click) 처리 함수 : click() 함수 사용 (클릭 이벤트 함수라고 칭합니다.)

### 반환된 값은 0번쨰 태그 정보 자체..(리스트 아님..)
movie_elements[0]

### 클릭 이벤트 처리 : 클릭 시키기
movie_elements[0].click()

In [7]:
### 다시 목록 페이지로 나오기(이전 페이지 나오기)
# - 웹브라우저의 history 정보를 이용합니다.
# - history 정보 : 페이지를 클릭하면서 이동한 이력정보를 가지고 있는 변수(브라우저가 관리)
# - execute_script() : javascript 프로그램 처리 함수 (javascript 프로그램 코드 작성 가능)
# - html의 이벤트 처리는 javascript 프로그램으로 처리됩니다.

### window.history.go(-1)
# - window : html 페이지가 보이는 브라우저 자체
# - history : 페이지 전환에 대한 이력 정보
# - go() : 이동(이전 또는 다음), -기호는 이전, -1은 이전페이지, -2는 이전-이전 페이지
driver.execute_script("window.history.go(-1)")

In [8]:
### 예외처리
# - 외부 연결 중 외부 물리적 연결에 문제가 발생한 경우 처리
# - 보통 윈도우 내부의 파일 읽어들이기, 웹클로링의 경우 접속하고자 하는 사이트가 중지된 경우 등등
# - 문제가 발생할 부분의 코드를 try 영역에 작성
#  -> 문제가 발생한 경우에 대한 처리 코드는 except 영역에 작성
try :
    # 1. 영화 목록 전체 정보를 추출
    #  -> 클릭 할 수 있는 태그를 찾아서 수집해야 합니다
    # #movieList > li > div.movie-list-info
    # div.movie-list-info
    
    ### 영화 목록 전체 가지고 오기 위한 태그 경로 정의
    movie_list_path = "div.movie-list-info"
    
    ### 태그 경로에 대한 태그 정보 조회하기
    # - find_elements 또는 find_element의 조회 결과 내에는 태그가 안보임
    #   (각 태그의 메모리 주소 정보만 가지고 있음)
    movie_elements = driver.find_elements(By.CSS_SELECTOR, "div.movie-list-info")
    print(len(movie_elements))
    
except :
    print("예외가 발생 했습니다. : find_elemenets()의 결과값을 가져오지 못했습니다.")
    movie_elements = []

20


In [9]:
if len(movie_elements) >= 1 :
    ### 클릭 이벤트 처리하기
    movie_elements[0].click()

else :
    print("수집된 영화 목록이 없습니다.")

In [10]:
### 웹브라우저 종료하기
driver.quit()

In [ ]:
# div.movie-list-info

### 영화목록 하단의 [더보기] 버튼 처리

In [ ]:
"""
<더보기>
 - [더보기] 버튼을 클릭 할 때 마다, 목록이 20개씩 계속 나옵니다.
 - [더보기] 버튼이 안보이는 시점에 -> 모든 영화 목록 전체를 가지고 오며,
   한번에 모든 영화목록 태그정보를 수집할 수 있음(방법적인 부분으로, 분석가 마음)

<더보기 버튼 처리 순서>
 1. [더보기] 버튼의 태그 정보를 추출하기 위한 태그 경로(패턴) 찾기
 2. 찾은 태그를 이용해서 -> [더보기] 클릭 처리하기
    -> [더보기] 버튼이 보이지 않을때까지 계속 클릭 처리
 3. [더보기] 버튼이 안보이는 시점에서 -> 모든 목록이 보이게 됩니다.
    -> 모든 목록이 보이는 시점의 현재 페이지 정보를 인식(현재 페이지 html 코드 수집) 해야 합니다.
       (클릭 또는 이전/이후에 대한 처리 후에는 전환 후 페이지 html 코드를 다시 인식해야함)
"""

In [12]:
### 크롬브라우저 open하는 코드를 함수로 만들어주세요...
# - url을 받아서 요청 및 응답받는 처리까지 모두 수행하는 함수로 만들어 주세요...
# - 드라이버 변수를 반환
def getDriver(url) :
    # 크롬브라우저 오픈
    driver = webdriver.Chrome()
    #웹 url 
    driver.get(url)
    return driver

In [13]:
driver = getDriver("https://www.megabox.co.kr/movie")

In [14]:
# 1. [더보기] 버튼의 태그 정보를 추출하기 위한 태그 경로(패턴) 찾기
# [더보기] 태그 경로 : #btnAddMovie
see_more = "#btnAddMovie"

### [더보기] 태그 정보 추출해 주세요.
# 변수명 : see_more_elements
see_more_element = driver.find_element(By.CSS_SELECTOR, "#btnAddMovie")

In [15]:
### 클릭 이벤트 한번 처리해 보기
# - [더보기] 버튼이 안보이는 상태에서, 아래 클릭 코드를 실행하면 오류 발생
try :
    see_more_element.click()
    
except :
    print("[더보기] 버튼이 더 이상 보이지 않습니다.")

In [ ]:
driver.execute_script("window.history.go(-1)")

In [ ]:
### 웹브라우저 종료하기
driver.quit()

### [더보기] 버튼 모두 펼쳐 놓는 코드 작성해주세요

In [15]:
### [더보기] 버튼이 안보일 때 까지 클릭해서 모든 목록을 펼쳐 놓기 위한
# -> 코드를 작성해 주세요.

cnt = 0
def see_more_all() :
    while True :
        try :
            see_more = "#btnAddMovie"
            see_more_element = driver.find_element(By.CSS_SELECTOR, see_more)
            ### 더보기 클릭
            see_more_element.click()
            ### 브라우저에서 클릭 후 로딩하는 시간 동안 프로그램은 잠시 대기 처리
            # - 1초 대기
            time.sleep(1)
    
            # ### cnt### 총 반복횟수 출력
            # cnt += 1
            
        except :
            print("[더보기] 버튼이 더 이상 보이지 않습니다.")
            break

In [16]:
see_more_all()

[더보기] 버튼이 더 이상 보이지 않습니다.


### 모든 목록이 보이는 시점의 현재 페이지 정보를 인식(현재 페이지 html 코드 수집)해야 합니다
##### (클릭 또는 이전/이후에 대한 처리 후에는 전환 후 페이지 html 코드를 다시 인식해야함)

In [ ]:
### window_handles : 페이지 변경 사항에 대한 정보를 담고 있습니다.
# - 클릭 후 페이지 변경이 되면, 해당 window_handles 리스트에 정보가 하나씩 추가됩니다.
# - 마지막 리스트 값을 사용하면 됩니다.
# - window_handles의 결과값은 리스트 타입
page_handle = driver.window_handles[-1]
page_handle

### window_handles의 마지막 값을 브라우저, 즉 window에 다시 반영 시켜야 합니다.
driver.switch_to.window(page_handle)

In [17]:
### 위 코드를 함수로 정의해서 사용하기
# - 함수 이름 : setPage
def setWindowPage(p_driver) :
    page_handle = p_driver.window_handles[-1]
    p_driver.switch_to.window(page_handle)
    time.sleep(1)

### [더보기] 이후 전체 목록 정보 조회하기

In [16]:
try :
    # 1. 영화 목록 전체 정보를 추출
    #  -> 클릭 할 수 있는 태그를 찾아서 수집해야 합니다
    
    ### 영화 목록 전체 가지고 오기 위한 태그 경로 정의
    movie_list_path = "div.movie-list-info"
    
    ### 태그 경로에 대한 태그 정보 조회하기
    # - find_elements 또는 find_element의 조회 결과 내에는 태그가 안보임
    #   (각 태그의 메모리 주소 정보만 가지고 있음)
    movie_elements = driver.find_elements(By.CSS_SELECTOR, "div.movie-list-info")
    print(len(movie_elements))
    
except :
    print("예외가 발생 했습니다. : find_elements()의 결과값을 가져오지 못했습니다.")
    movie_elements = []

40


### 영화목록 1개를 기준으로 샘플링 하기

In [19]:
### 영화목록 1개 추출하기, 클릭해서 정보 페이지로 전환하기
movie_elements[4].click()

In [ ]:
### 페이지 전환 이후 전환된 페이지를 window(브라우저)에 인식 시키기

In [20]:
setWindowPage(driver)

In [ ]:
### 영화 상세 정보 추출하기
# : 영화제목, 실관람평점, 예매순위, 예매율, 누적관객수
# : 영화제목, 개별관람평점, 개별관란명내용, 긍정(관람평점6이상)/부정(관렴평점6미만)

In [21]:
### 영화 상세 정보 추출하기
# - [영화 제목] 추출하기
# - 영화제목이 있는 태그 경로 정의하기
title_path = "p.title"

##3 영화제목이 있는 태그 정보 추출하기
title_element = driver.find_element(By.CSS_SELECTOR, title_path)

### 추출한 태그에서 영화제목 text 추출하여 출력하기
title = title_element.text
print(f"영화제목 : {title}")

영화제목 : 캡틴 아메리카: 브레이브 뉴 월드


In [22]:
### [실관람평] 추출하기
# - 실관람평 태그 경로 정의
score_path = "#mainMegaScore > p > em"

# 실관람평 태그 추출하기
score_element = driver.find_element(By.CSS_SELECTOR, score_path)

# - 실관람평 텍스트 추출하기
score = score_element.text
print(f"실관람평 : {score}")

실관람평 : 8.3


In [23]:
### [예매순위] 및 [예매율] 추출하기
### [예매순위] 추출하기
# - 예매순위 태그 경로 정의 : ranking
ranking_path = "div.info > div.rate > p.cont > em"

# 예매순위 태그 추출하기
ranking_element = driver.find_element(By.CSS_SELECTOR, ranking_path)

# - 예매순위 텍스트 추출하기
ranking = ranking_element.text

### [예매율] 추출하기
# - 예매율 태그 경로 정의 : rate
#  -> 예매율은 selector가 안될겁니다. (selector 되는 시점의 태그를 찾아야 합니다.)
# - hint : 문자열.split("구분자") 함수 사용 -> 함수 결과는 구분된 값을 리스트(list)로 반환
rate_path = "div.info > div.rate > p.cont"

# 예매율 태그 추출하기
rate_element = driver.find_element(By.CSS_SELECTOR, rate_path)

# - 예매율 텍스트 추출하기
rate_text = rate_element.text
rate = rate_text.split()[1][1:4]

print(f"예매순위 : {ranking}위, 예매율 : {rate}%")

예매순위 : 5위, 예매율 : 4.7%


In [24]:
### [예매순위] 및 [예매율] 추출하기
temp_path = "div.info > div.rate > p.cont"
temp_element = driver.find_element(By.CSS_SELECTOR, temp_path)
temp_text = temp_element.text
rate = temp_text.split(" ")[1][1:4]
ranking = temp_text.split(" ")[0][0:-1]

print(f"예매순위 : {ranking}위, 예매율 : {rate}%")

예매순위 : 5위, 예매율 : 4.7%


In [25]:
### [누적관객수] 추출하기
# - 누적관객수 태그 경로 정의 : audience
# - 최종 값에서 콤마(,)는 제외하고 숫자만 추출하여 변수에 저장하기
audience_path = "div.info > div.audience > p > em"

# 누적관객수 태그 추출하기
audience_element = driver.find_element(By.CSS_SELECTOR, audience_path)

# - 누적관객수 텍스트 추출하기
audience = audience_element.text.replace(',','')
print(f"누적관객수 : {audience}명")

누적관객수 : 1312372명


### [실관람평] 탭을 클릭하여 개별 관련 평점 및 평내용 추출하기

In [26]:
### [실관람평] 탭 클릭을 위한 태그 정보 추출하기
try :
    movie_tab_path = "#contentData > div:nth-child(5) > div.tab-list.fixed > ul"
    movie_tab_element = driver.find_element(By.CSS_SELECTOR, movie_tab_path)
    ### 더보기 클릭
    movie_tab_element.click()
            
except :
    print("[더보기] 버튼이 더 이상 보이지 않습니다.")

In [29]:
setWindowPage(driver)

In [ ]:
### [실관람평] 탭 클릭을 위한 태그 정보 추출하기
# - 탭 경로 정의
movie_tab_path = "#contentData > div:nth-child(5) > div.tab-list.fixed > ul > li:nth-child(2) > a"

# - 탭 태그 정보 추출

movie_tab_element = driver.find_element(By.CSS_SELECTOR,movie_tab_path)
# - 탭 태그 텍스트 추출해 보기

print(movie_tab_element.text)
# - 탭 태그의 링크 주소(url) 추출해 보기
# --> a = {href:"url"}
# --> BeautifulSoup 방식 : 태그변수["href"]
# - 속성의 값 추출하는 함수 : 태그변수.get_attribute("속성명")

print(movie_tab_element.get_attribute("href"))

In [ ]:
### [실관람평] 탭 클릭 처리하기
movie_tab_element.click()

In [ ]:
### 클릭 이벤트 처리 후에는 브라우저에 현재 페이지 정보 인식시키기
setWindowPage(driver)

In [35]:
### [개별 관람평점] 추출하기
# - 개별관람평점 태그 경로 정의
use_score_path = "div.story-box > div > div.story-cont > div.story-point > span"
# - 개별관람평점이 있는 모든 태그 정보 추출하기
use_score_elements = driver.find_elements(By.CSS_SELECTOR, use_score_path)
print("개별관람평점 전체 태그 조회 갯수 : ", len(use_score_elements))

# - 0번째 태그의 텍스트 값 추출하여 출력해보기
use_score = use_score_elements[9].text
print(f"0번째 개별관람평 평점 : {use_score}")

개별관람평점 전체 태그 조회 갯수 :  10
0번째 개별관람평 평점 : 10


In [49]:
### [개별 관람평내용] 추출하기
# - 개별관람평내용 태그 경로 정의
use_comment_path = "div.story-area > div.story-box > div > div.story-cont > div.story-txt"
# - 개별관람평내용 있는 모든 태그 정보 추출하기
use_comment_elements = driver.find_elements(By.CSS_SELECTOR, use_comment_path)
print("개별관람평점 전체 댓글 조회 갯수 : ", len(use_comment_elements))

# - 0번째 태그의 텍스트 값 추출하여 출력해보기
use_comment = use_comment_elements[0].text
print(f"0번째 개별관람평 내용 : {use_comment}")

개별관람평점 전체 댓글 조회 갯수 :  10
0번째 개별관람평 내용 : 마블영화지만.재미는 별로 없네요.. 킬링타임용 영화이네요


In [50]:
### 현재 영화정보에 대한 수집이 완료되면,
# - 전체 목록 페이지로 돌아가기 : 이전페이지로 가기
driver.execute_script("window.history.go(-1)")
# - 이전페이지 로딩되는 시간동안 프로그램 실행 잠시 대기
time.sleep(1)

In [ ]:
### 전체 목록으로 이동하기
driver.execute_script("window.history.go(-1)")
time.sleep(1)

In [17]:
### 웹브라우저 종료하기
driver.quit()